# Example usage (last updated: 2023/6/5 zensvi=v0.4.0)

To use `zensvi` in a project:

In [1]:
#%pip install --upgrade zensvi # if zen-svi is not installed

import zensvi

print(zensvi.__version__)

from zensvi.download import GSVDownloader, MLYDownloader
from zensvi.cv import Segmenter
from zensvi.transform import ImageTransformer

0.4.0


# Download SVI
## Google Street View

In [ ]:
gsv_api_key = "YOUR_OWN_GSV_API_KEY" # please register your own GSV API key at https://developers.google.com/maps/documentation/streetview/get-api-key
network_type = "all" # network type for OSMnx
custom_filter = None # custom filter for OSMnx
gsv_downloader = GSVDownloader(gsv_api_key = gsv_api_key, # GSV API key for metadata augmentation
                            distance=1, # interval distance between each sample point
                            grid = False, # if True, sample points are generated in a grid pattern
                            grid_size = 1) # grid size in meters
gsv_downloader.download_svi("path/to/output", # output directory
                        path_pid = None, # if you already have a list of panorama IDs, you can specify the path to the file here
                        zoom=2, # zoom level of the image (i.e. resolution)
                        h_tiles=4, # number of horizontal tiles
                        v_tiles=2, # number of vertical tiles
                        cropped=False, # if True, only the upper half of the image is saved
                        full=True, # if True, the full image is saved. If False, the image tiles are separately saved
                        lat=None, lon=None, # latitude and longitude of the location to download
                        input_csv_file="", # path to the input CSV file containing the location information
                        input_shp_file = "", # path to the input shapefile containing the location information
                        input_place_name = "", # name of the location to download
                        id_columns=None, # column name of the ID in the input CSV file or shapefile
                        buffer = 0, # buffer size in meters around the input location
                        augment_metadata=False, # if True, metadata is augmented using the Google Maps API
                        update_pids = False, # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
                        network_type = network_type, # network type for OSMnx
                        custom_filter = custom_filter) # custom filter for OSMnx

## Mapillary

In [ ]:
mly_api_key = "YOUR_OWN_MLY_API_KEY" # please register your own Mapillary API key at https://www.mapillary.com/dashboard/developers
# additional keyword arguments for the Mapillary API
kwarg = {"fields": ["all"], # A list of options, either as [‘all’], or a list of fields.
        "zoom": 14, # The zoom level of the tiles to obtain, defaults to 14
        "radius": 50, #The radius of the images obtained from a center center
        "image_type": "both", # The tile image_type to be obtained, either as ‘flat’, ‘pano’ (panoramic), or ‘both’.
        "min_captured_at": "2016", # The min date. Format from ‘YYYY’, to ‘YYYY-MM-DDTHH:MM:SS’
        "max_captured_at": "2020", # The max date. Format from ‘YYYY’, to ‘YYYY-MM-DDTHH:MM:SS’
        "org_id": None, # The organization id, ID of the organization this image (or sets of images) belong to. It can be absent. Thus, default is -1 (None)
        }
mly_downloader = MLYDownloader(mly_api_key, # Mapillary API key
                                log_path = None, # path to the log file
                                distance = 1, # interval distance between each sample point
                                grid = False, # if True, sample points are generated in a grid pattern
                                grid_size = 1) # grid size in meters
mly_downloader.download_svi("path/to/output", # output directory
                        path_pid = None, # if you already have a list of panorama IDs, you can specify the path to the file here
                        lat=None, lon=None, # latitude and longitude of the location to download
                        input_csv_file="", # path to the input CSV file containing the location information
                        input_shp_file = "", # path to the input shapefile containing the location information
                        input_place_name = "", # name of the location to download
                        id_columns=None, # column name of the ID in the input CSV file or shapefile
                        buffer = 0, # buffer size in meters around the input location
                        update_pids = False, # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
                        resolution = 1024, # resolution of the image
                        cropped = False, # if True, only the upper half of the image is saved
                        network_type = network_type, # network type for OSMnx
                        custom_filter = custom_filter # custom filter for OSMnx
                        **kwarg)

# Segmentation
## Cityscapes

In [ ]:
# initialize the segmenter
segmenter  = Segmenter()

# set arguments
dir_input = "path/to/input" 
dir_image_output = "path/to/image_output"
dir_pixel_ratio_output = "path/to/pixel_ratio_output"
save_image_options = ["segmented_image", "blend_image"] # segmented_image (colored image), blend_image (blended image)
pixel_ratio_save_format = ["csv", "json"]
segmenter.segment(dir_input, 
                dir_image_output = dir_image_output,
                dir_pixel_ratio_output = dir_pixel_ratio_output,
                save_image_options = save_image_options,
                pixel_ratio_save_format = pixel_ratio_save_format)

## Mapillary Vistas

In [ ]:
# initialize the segmenter
model_name="facebook/mask2former-swin-large-mapillary-vistas-semantic"
dataset="mapillary"
segmenter  = Segmenter(model_name=model_name, dataset=dataset)

# set arguments
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_pixel_ratio_output = "path/to/pixel_ratio_output"
save_image_options = ["segmented_image", "blend_image"]
pixel_ratio_save_format = ["csv", "json"]
segmenter.segment(dir_input, 
                dir_image_output = dir_image_output,
                dir_pixel_ratio_output = dir_pixel_ratio_output,
                save_image_options = save_image_options,
                pixel_ratio_save_format = pixel_ratio_save_format)

# Transformation: perspective and Fisheye

In [ ]:
dir_input="path/to/input"
dir_output="path/to/output"
image_transformer = ImageTransformer(dir_input=dir_input, dir_output=dir_output)
image_transformer.transform_images(style_list=["perspective", "equidistant_fisheye", "orthographic_fisheye", "stereographic_fisheye", "equisolid_fisheye"], # list of projection styles
                                    FOV = 90, # field of view
                                    aspects = (9, 16), # aspect ratio
                                    show_size=100) # size of the image to show (i.e. scale factor)